# Restaurant Name

In [1]:
# Load env variables
from dotenv import load_dotenv

load_dotenv()

True

In [14]:
from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(model="gemini-2.0-flash")

# Simple text invocation
result = llm.invoke("What is the capital of France? Provide only the name of the city.")
print(result.content)

Paris



In [5]:
from langchain.prompts import PromptTemplate

restaurant_name_prompt_template = PromptTemplate(
    input_variables=["cuisine"],
    template="I want to open a restaurant for {cuisine} cuisine. Suggest a good restaurant name. Response must include only the name of the restaurant.",
)

restaurant_name_prompt_template_formatted = restaurant_name_prompt_template.format(cuisine="Chinese")
restaurant_name_prompt_template_formatted

'I want to open a restaurant for Chinese cuisine. Suggest a good restaurant name. Response must include only the name of the restaurant.'

In [7]:
from langchain.chains import LLMChain

restaurant_name_chain = LLMChain(
    prompt=restaurant_name_prompt_template, llm=llm
)

name_of_restaurant = restaurant_name_chain.invoke("Chinese")

print(name_of_restaurant)

{'cuisine': 'Chinese', 'text': 'Golden Dragon\n'}


# Simple Sequential Chain

In [8]:
food_items_prompt_template = PromptTemplate(
    input_variables=["restaurant_name"],
    template="Provide three items as comma separated list to include in a menu for a restaurant called {restaurant_name}. Response must include only the items.",
)

food_items_chain = LLMChain(
    prompt=food_items_prompt_template, llm=llm, verbose=True
)

In [11]:
from langchain.chains import SimpleSequentialChain

overall_chain = SimpleSequentialChain(
    chains=[restaurant_name_chain, food_items_chain], verbose=True
)

menu_items = overall_chain.invoke("Chinese")

print(f"{menu_items['input']}:")
print({menu_items['output']})



> Entering new SimpleSequentialChain chain...
Golden Dragon



> Entering new LLMChain chain...
Prompt after formatting:
Provide three items as comma separated list to include in a menu for a restaurant called Golden Dragon
. Response must include only the items.

> Finished chain.
Peking Duck, Kung Pao Chicken, Spring Rolls


> Finished chain.
Chinese:
{'Peking Duck, Kung Pao Chicken, Spring Rolls\n'}


# Sequential Chain

In [12]:
from langchain.chains import SequentialChain

restaurant_name_chain = LLMChain(
    prompt=restaurant_name_prompt_template, llm=llm, output_key="restaurant_name"
)

food_items_chain = LLMChain(
    prompt=food_items_prompt_template, llm=llm, verbose=True, output_key="food_items"
)

seq_chain = SequentialChain(chains=[restaurant_name_chain, food_items_chain], input_variables=["cuisine"], output_variables=["restaurant_name", "food_items"], verbose=True)

seq_chain({"cuisine": "American"})

d:\Documents\GitHub\restaurant-name-generator\.venv\Lib\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(




> Entering new SequentialChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
Provide three items as comma separated list to include in a menu for a restaurant called The Blue Plate Special
. Response must include only the items.

> Finished chain.

> Finished chain.


{'cuisine': 'American',
 'restaurant_name': 'The Blue Plate Special\n',
 'food_items': 'Meatloaf, Mashed Potatoes, Green Beans\n'}